In [1]:
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
from PIL import Image
import numpy as np

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.images = [f for f in os.listdir(img_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.images[idx])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image

def calculate_mean_std(dataset_path, batch_size=32, num_workers=0):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    dataset = CustomDataset(dataset_path, transform=transform)
    loader = DataLoader(dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False)

    mean = 0.0
    std = 0.0
    num_samples = 0

    for images in loader:
        batch_samples = images.size(0)
        images = images.view(batch_samples, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        num_samples += batch_samples

    mean /= num_samples
    std /= num_samples

    return mean, std



In [4]:
dataset_path = r"Data_Split\train_set\A"
mean, std = calculate_mean_std(dataset_path)
print(f"Dataset mean: {mean}")
print(f"Dataset std: {std}")

Dataset mean: tensor([0.5514, 0.4094, 0.3140])
Dataset std: tensor([0.1299, 0.1085, 0.0914])


In [8]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

class DualTaskDataset(Dataset):
    def __init__(self, image_dir, mask_dir, skeleton_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.skeleton_dir = skeleton_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpeg') or f.endswith('.jpg') or f.endswith('.png')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        
        # Correct image path construction
        img_path = os.path.join(self.image_dir, img_name)

        # Construct the mask and skeleton paths using the same filename (replace extension)
        base_name = os.path.splitext(img_name)[0]
        mask_name = base_name + '.png'
        skeleton_name = base_name + '.png'

        # Full paths for mask and skeleton
        mask_path = os.path.join(self.mask_dir, mask_name)
        skeleton_path = os.path.join(self.skeleton_dir, skeleton_name)

        # Load the images
        image = Image.open(img_path).convert('RGB')
        mask = Image.open(mask_path).convert('L')
        skeleton = Image.open(skeleton_path).convert('L')

        # Apply transforms, if any
        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)
            skeleton = self.transform(skeleton)

        return image, mask, skeleton

def get_dataloader(image_dir, mask_dir, skeleton_dir, batch_size=32, num_workers=0):
    # Define transforms
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5514, 0.4094, 0.3140], std=[0.1299, 0.1085, 0.0914])
    ])

    # Create dataset
    dataset = DualTaskDataset(image_dir, mask_dir, skeleton_dir, transform=transform)

    # Create dataloader
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

    return dataloader


In [9]:
# Usage example
if __name__ == "__main__":
    image_dir = "DATA/IMG"
    mask_dir = "DATA/MASKS"
    skeleton_dir = "DATA/Skeleton"

    dataloader = get_dataloader(image_dir, mask_dir, skeleton_dir)

    # Iterate through the dataloader
    for images, masks, skeletons in dataloader:
        print(f"Batch shapes: Images {images.shape}, Masks {masks.shape}, Skeletons {skeletons.shape}")
        # Your training loop here
        break

FileNotFoundError: [Errno 2] No such file or directory: 'DATA/MASKS\\MAIZEROOTS2016_T015_L029_2016.08.31_122038_004_JM.png'

In [6]:
image_path = r"Data_Split\val\B\19MRTUBES3001TO3999CAM4_T131_L037_2019.07.31_090152_001_BI.png"

In [17]:
image = Image.open(image_path)
width, height = image.size

print(f"Image dimensions: {width}x{height}")

Image dimensions: 640x480


In [8]:
import cv2

img = cv2.imread(image_path)

img_np = np.asarray(img)

print("img_np.shape: ", img_np.shape)

img_np.shape:  (480, 640, 3)
